In [1]:
!pip install yfinance
!pip install pandas_ta
!pip install scikit-learn
!pip install matplotlib
!pip install plotly
!pip install "notebook>=5.3" "ipywidgets>=7.5"
!pip install bokeh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 13.2 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 kB 4.2 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.8-cp310-cp310-linux_x86_64.whl size=300806 sha256=86fb9b53c797c3d650f0388b2cd4ec043c6b5f7e20c05f1419e2c6e74fcd61d4
  Stored in directory: /root/.cache/pip/wheels/75/79/e5/8838db0594cc6c587142fd2563356392ade6255c5930411069
Successfully built peewee
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=d8e68614c0a5158f7bc66886661184e9f8229ece0d1236e63e913e665d21d31e
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/__script__.py
/kaggle/input/__results__.html
/kaggle/input/input_requirements.txt
/kaggle/input/__script__.ipynb
/kaggle/input/__output__.json
/kaggle/input/custom.css


In [3]:
import pandas as pd
import yfinance as yf

# Load data
df = yf.Ticker('^GSPC').history(period='max', interval='1d', end='2024-12-31')
df = df[['Open', 'High', 'Low', 'Close']].dropna()  # Select relevant columns and drop NA

# Prepare target column
df['Target'] = df['Close'].shift(-1)
df.dropna(inplace=True)

# Display data
df.tail()

X = df[['Open', 'High', 'Low', 'Close']].values  # Feature matrix
y = df['Target'].values.reshape(-1, 1)          # Target column reshaped

In [4]:
from sklearn.preprocessing import MinMaxScaler

# Define split ratio and calculate split index
split_ratio = 0.8
split_index = int(len(X) * split_ratio)

# Split into training and test sets
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Initialize scaler
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

# Fit the scalers on training data
scaler_X.fit(X_train)  # Fit on X_train
scaler_y.fit(y_train)  # Fit on y_train

X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)

y_test = scaler_y.transform(y_test)
y_train = scaler_y.transform(y_train)


print("y_train_scaled range: min =", X_train.min(), ", max =", X_train.max())
print("y_test_scaled range: min =", X_test.min(), ", max =", X_test.max())
print("y_train_scaled range: min =", y_train.min(), ", max =", y_train.max())
print("y_test_scaled range: min =", y_test.min(), ", max =", y_test.max())

y_train_scaled range: min = 0.0 , max = 1.0
y_test_scaled range: min = 0.4374925663865714 , max = 4.012773692921709
y_train_scaled range: min = 0.0 , max = 1.0
y_test_scaled range: min = 0.441302408626035 , max = 3.995817745790438


In [5]:
# Parameters
backcandles = 30  # Number of past timesteps to use

def create_sliding_windows(X_scaled, y_scaled, backcandles):
    """
    Create sliding windows for features (X) and target (y).
    """
    X = []
    y = []
    for i in range(backcandles, len(X_scaled)):
        X.append(X_scaled[i - backcandles:i])  # Append `backcandles` rows
        y.append(y_scaled[i])  # Append the target at the end of the window
    return np.array(X), np.array(y)

# Create sliding windows for training and testing sets
X_train, y_train = create_sliding_windows(X_train, y_train, backcandles)
X_test, y_test = create_sliding_windows(X_test, y_test, backcandles)

# Print shapes to verify
print("X_train shape:", X_train.shape)  # (samples_train, backcandles, num_features)
print("y_train shape:", y_train.shape)  # (samples_train, 1)
print("X_test shape:", X_test.shape)    # (samples_test, backcandles, num_features)
print("y_test shape:", y_test.shape)    # (samples_test, 1)

# Example: print the first sliding window sample
print("\nFirst sample of X_train (features):")
print(X_train[0])

print("\nFirst target value of y_train:")
print(y_train[0])

X_train shape: (19462, 30, 4)
y_train shape: (19462, 1)
X_test shape: (4843, 30, 4)
y_test shape: (4843, 1)

First sample of X_train (features):
[[0.01156168 0.00856329 0.00875791 0.00870616]
 [0.01162715 0.00862787 0.00882396 0.00877181]
 [0.01160096 0.00860204 0.00879754 0.00874555]
 [0.01148966 0.00849225 0.00868526 0.00863393]
 [0.01156168 0.00856329 0.00875791 0.00870616]
 [0.01145693 0.00845996 0.00865223 0.00860111]
 [0.01137182 0.00837601 0.00856637 0.00851575]
 [0.01135873 0.00836309 0.00855316 0.00850262]
 [0.01143729 0.00844059 0.00863242 0.00858141]
 [0.0115093  0.00851163 0.00870507 0.00865363]
 [0.01131945 0.00832435 0.00851353 0.00846323]
 [0.01132599 0.0083308  0.00852014 0.00846979]
 [0.01129981 0.00830497 0.00849372 0.00844353]
 [0.01137837 0.00838247 0.00857298 0.00852232]
 [0.01144383 0.00844705 0.00863902 0.00858797]
 [0.01154858 0.00855038 0.0087447  0.00869303]
 [0.01159441 0.00859558 0.00879093 0.00873899]
 [0.01147002 0.00847288 0.00866544 0.00861424]
 [0.01154

In [6]:
from keras.models import Model
from keras.layers import Input, LSTM, Dropout, Dense

# Define the input layer
lstm_input = Input(shape=(X_train.shape[1], X_train.shape[2]), name='lstm_input')

# Add the first LSTM layer with Dropout
x = LSTM(units=64, activation='relu', name='lstm_layer_1')(lstm_input)
x = Dropout(0.2)(x)

# Add the Dense output layer
output = Dense(units=1, activation='linear', name='output_layer')(x)

# Create the model
model = Model(inputs=lstm_input, outputs=output, name='LSTM_Model')

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mape'])

# Display the model summary
model.summary()

Model: "LSTM_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_input (InputLayer)         │ (None, 30, 4)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_layer_1 (LSTM)             │ (None, 64)             │        17,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,729 (69.25 KB)

 Trainable params: 17,729 (69.25 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=32, batch_size=15,
    callbacks=[early_stopping]
)

Epoch 1/32


I0000 00:00:1736424632.066142     144 service.cc:145] XLA service 0x7f9c9c0b8ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1736424632.066200     144 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1736424632.066204     144 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  36/1298 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0438 - mape: 171.8316

I0000 00:00:1736424634.695398     144 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1298/1298 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 0.0045 - mape: 2183.3545
Epoch 2/32
1298/1298 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 8.5448e-04 - mape: 383.0601
Epoch 3/32
1298/1298 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 7.6582e-04 - mape: 197.0859
Epoch 4/32
1298/1298 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 7.4869e-04 - mape: 377.2030
Epoch 5/32
1298/1298 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 8.1803e-04 - mape: 279.6949
Epoch 6/32
1298/1298 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 7.8913e-04 - mape: 2059.4075
Epoch 7/32
1298/1298 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 7.8604e-04 - mape: 615.0439
Epoch 8/32
1298/1298 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 7.4593e-04 - mape: 139.2961
Epoch 9/32
1298/1298 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 7.5383e-04 - mape: 2892.4014
Epoch 10/32
1298/1298 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 7.8386e-04 - mape: 147.8171
Epoch 11/32
1298/1298 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 7.6644e-04 - mape: 107.5985
Epoch 12/32
1298/12

In [8]:
from sklearn.metrics import mean_absolute_percentage_error, max_error, mean_squared_error
import numpy as np

# Predict scaled values
y_pred=model.predict(X_test)
y_pred = y_pred.flatten()
y_test = y_test.flatten()

print(y_pred.shape,y_test.shape)
# # Inverse transform the predictions and actual target values
# y_pred = scaler_y.inverse_transform(y_pred)
# y_test = scaler_y.inverse_transform(y_test)  # Inverse transform y_test

# Calculate performance metrics
mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # Convert to percentage
mse = mean_squared_error(y_test, y_pred)

# Replace zeros in y_actual to avoid division by zero
y_test_safe = np.where(y_test == 0, 1e-6, y_test)

# Compute percentage deviations
percentage_deviation = np.abs((y_test - y_pred) / y_test_safe) * 100

# Find the maximum percentage deviation and its index
mpd_index = np.argmax(percentage_deviation)
mpd = percentage_deviation[mpd_index]

# Print the metrics
print(f"MAPE: {mape:.4f}%")
print(f"MPD (Maximum Percentage Deviation): {mpd:.4f}%")
print(f"MSE: {mse:.8f}")

# Print the true and predicted values at the point of maximum deviation
print(f"\nPoint of Maximum Deviation:")
print(f"True Value: {y_test[mpd_index]}")
print(f"Predicted Value: {y_pred[mpd_index]}\n")

# Print first 5 pairs of true and predicted values
for i in range(5):
    print(f"Actual: {y_test[i]}, Predicted: {y_pred[i]}")

152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
(4843,) (4843,)
MAPE: 4.7090%
MPD (Maximum Percentage Deviation): 27.4838%
MSE: 0.03367952

Point of Maximum Deviation:
True Value: 3.9628380983962694
Predicted Value: 2.873699188232422

Actual: 0.8025291075215921, Predicted: 0.8022464513778687
Actual: 0.7944992329380677, Predicted: 0.8062026500701904
Actual: 0.7826284225172548, Predicted: 0.8075615167617798
Actual: 0.7794111989410056, Predicted: 0.803911566734314
Actual: 0.7823067081744367, Predicted: 0.7943788766860962


In [9]:
from bokeh.plotting import figure, show, output_notebook
import numpy as np

y_pred = np.squeeze(y_pred)
y_test = np.squeeze(y_test)

# Enable Bokeh output in the notebook
output_notebook()

x = np.arange(len(y_test))   # Sample numbers from 1 to 100

# Create a figure
p = figure(title="Test vs Predicted Values (Interactive)",
           x_axis_label='Sample',
           y_axis_label='Value',
           width=800,
           height=400)

# Add Test line
p.line(x, y_test, legend_label="Test", line_width=2, color="black")
# Add Predicted line
p.line(x, y_pred, legend_label="Predicted", line_width=2, color="red")

# Show the plot
show(p)

Loading BokehJS ...